# 第8章 チャットボット

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一導入" data-toc-modified-id="一、導入">一、導入</a></span></li>
        <li>
        <span><a href="#二アイデンティティとコンテキストの構築" data-toc-modified-id="二、アイデンティティとコンテキストの構築">二、アイデンティティとコンテキストの構築</a></span></li><li>
        <span><a href="#三注文ボット" data-toc-modified-id="三、注文ボット">三、注文ボット</a></span>
        <ul class="toc-item">
            <li><span><a href="#31-json要約の作成" data-toc-modified-id="3.1 json要約の作成">3.1 JSON要約の作成</a></span></li>
        </ul>
        </li>
    </ul>
</div>

## 一、導入

大規模言語モデルを使用する際の興奮すべきことの一つは、非常に少ない作業量でカスタマイズされたチャットボット（Chatbot）を構築できることです。本節では、チャット形式を利用して、個人化された（または特定のタスクや行動に特化した）チャットボットとの拡張対話を行う方法を探索します。

In [ ]:
import openai
# サードパーティライブラリのインポート

openai.api_key = "sk-..."
# API_KEYの設定、ご自身のAPI_KEYに置き換えてください

ChatGPTのようなチャットモデルは実際には、一連のメッセージを入力として受け取り、モデルが生成したメッセージを出力として返すように組み立てられています。このチャット形式は元々複数ターンの対話を簡単にするために設計されましたが、これまでの学習を通じて分かるように、対話を伴わない**単一ターンタスク**にも同様に有用です。

## 二、アイデンティティとコンテキストの構築

次に、2つの補助関数を定義します。

最初のメソッドは、チュートリアル全体を通してあなたと共にあった```get_completion```で、単一ターン対話に適用されます。プロンプトを**ユーザーメッセージ**のような対話ボックスに配置します。もう一つは```get_completion_from_messages```と呼ばれ、メッセージリストを渡します。これらのメッセージは大量の異なる**役割**（roles）から来ることができ、これらの役割について説明します。

最初のメッセージでは、システムアイデンティティでシステムメッセージ（system message）を送信し、これは全体的な指示を提供します。システムメッセージはアシスタントの行動と役割を設定し、対話の高レベル指示として機能します。アシスタントの耳元でささやき、その応答を導くものと想像できますが、ユーザーはシステムメッセージに気づきません。そのため、ユーザーとして、もしあなたがChatGPTを使用したことがあるなら、ChatGPTのシステムメッセージが何であるかを知らないかもしれませんが、これは意図的なものです。システムメッセージの利点は、開発者にリクエスト自体を対話の一部にすることなく、アシスタントを導き、その応答を指導する方法を提供することです。

ChatGPTのウェブインターフェースでは、あなたのメッセージはユーザーメッセージと呼ばれ、ChatGPTのメッセージはアシスタントメッセージと呼ばれます。しかし、チャットボットを構築する際、システムメッセージを送信した後、あなたの役割はユーザー（user）としてのみ機能することもできますし、ユーザーとアシスタント（assistant）の間で交互に役割を果たし、対話コンテキストを提供することもできます。

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # モデル出力のランダム性を制御
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # モデル出力のランダム性を制御
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

今度は対話でこれらのメッセージを使ってみましょう。上記の関数を使用してこれらのメッセージから得られる回答を取得し、同時により高い温度（temperature）を使用します（高いほどより多様性が生成されます、詳細は第7章を参照）。

システムメッセージは、あなたはシェイクスピアのように話すアシスタントだと言っています。これは**どのように振る舞うべきか**をアシスタントに説明する方法です。その後、最初のユーザーメッセージは*ジョークを教えて*です。次にアシスタントの身分で回答を与えるのは、*なぜ鶏は道路を渡ったのか？*最後にユーザーメッセージとして*分からない*を送信します。

In [4]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, fair sir.


In [ ]:
# 日本語
messages =  [  
{'role':'system', 'content':'あなたはシェイクスピアのように話すアシスタントです。'},    
{'role':'user', 'content':'ジョークを教えて'},   
{'role':'assistant', 'content':'なぜ鶏は道路を渡ったのか'},   
{'role':'user', 'content':'分からない'}  ]

In [9]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

因为它要去找“母鸡”。哈哈哈！（注：此为英文双关语，"chicken"是鸡的意思，也是胆小的意思；"cross the road"是过马路的意思，也是“破坏规则”的意思。）


（注：上記の例ではtemperature = 1を選択したため、モデルの回答は比較的ランダムで大きく異なります（創造性に富んだものも少なくありません）。ここに別の回答を付け加えます：

シェイクスピア風の詩でお答えしましょう：

鶏の心が前進を欲するとき、
道路の際がその選択となりけり。
車ゆっくりと進み天晴れて、
クラクション響き交錯せり。

なにゆえ何処へ行くやと問わん？
大道の上に征なき故に、
鶏は躍り進みて、
その決断に迷いなし。

鶏の智慧なにを語らん、
道路孤独にして漆の如し。
されども其の勇気人を嘆かせ、
勇往邁進して退くことなし。

故に鶏道路を渡る理由は？
車流騒々の困厄に耐えんため。
その鳴き声勇ましく一躍し、
誇らしき絵巻を成就せり。

故に笑話の妙処は、
鶏の勇気溢るる伴いなり。
人生の道を恐れず笑談し、
智あり勇ありて妙を尽くす。

このシェイクスピア風の回答があなたに少しの楽しみをもたらすことを願います！）

別の例を見てみましょう。アシスタントのメッセージは*あなたはフレンドリーなチャットボットです*、最初のユーザーメッセージは*こんにちは、私はIsaです*。最初のユーザーメッセージを取得したいと思います。

In [10]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's great to meet you. How can I assist you today?


In [ ]:
# 日本語
messages =  [  
{'role':'system', 'content':'あなたはフレンドリーなチャットボットです。'},    
{'role':'user', 'content':'こんにちは、私はIsaです。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

もう一つの例を試してみましょう。システムメッセージは、あなたはフレンドリーなチャットボットです、最初のユーザーメッセージは、はい、私の名前を思い出させてもらえますか？

In [12]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but since we don't have any personal information about you, I don't know your name. Can you please tell me your name?


In [ ]:
# 日本語
messages =  [  
{'role':'system', 'content':'あなたはフレンドリーなチャットボットです。'},    
{'role':'user', 'content':'はい、私の名前を思い出させてもらえますか？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

上記で見るように、モデルは実際には私の名前を知りません。

そのため、言語モデルとの各やり取りは相互に独立しており、これは現在の対話でモデルが参照できるように、すべての関連メッセージを提供する必要があることを意味します。モデルが対話の早期部分を参照または「記憶」してほしい場合は、モデルの入力に早期の交流を提供する必要があります。これをコンテキスト（context）と呼びます。以下の例を試してみてください。

In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


別の回答を付け加えます：

*あなたの名前はIsaです！どうして忘れることができるでしょうか？*

In [ ]:
# 日本語
messages =  [  
{'role':'system', 'content':'あなたはフレンドリーなチャットボットです。'},
{'role':'user', 'content':'こんにちは、私はIsaです'},
{'role':'assistant', 'content': "こんにちはIsa！お会いできて嬉しいです。今日はお手伝いできることはありますか？"},
{'role':'user', 'content':'はい、私の名前を思い出させてもらえますか？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

今度は、モデルに必要なコンテキスト、つまり以前の対話で言及された私の名前を提供しました。その後、同じ質問、つまり私の名前は何ですかと尋ねます。モデルは必要なすべてのコンテキストを持っているため、入力のメッセージリストで見るように応答できます。

## 三、注文ボット

今度は「注文ボット」を構築します。これがユーザー情報を自動的に収集し、ピザ店の注文を受け付ける必要があります。

以下の関数は、手動入力を避けるためにユーザーメッセージを収集します。この関数は、以下で構築するユーザーインターフェースからプロンプトを収集し、コンテキスト（```context```）というリストに追加し、モデルを呼び出すたびにそのコンテキストを使用します。モデルの応答もコンテキストに追加されるため、ユーザーメッセージとモデルメッセージの両方がコンテキストに追加され、コンテキストは徐々に長くなります。このようにして、モデルは次にすべきことを決定するために必要な情報を持ちます。

In [19]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

今度は、このUIを設定して実行し、注文ボットを表示します。初期のコンテキストには、メニューを含むシステムメッセージが含まれており、各呼び出し時に使用されます。その後、対話が進むにつれて、コンテキストも継続的に成長します。

In [ ]:
!pip install panel

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

実行結果はインタラクティブです。以下の日本語版をご覧ください。

### 3.1 JSON要約の作成

ここで、モデルに注文システムに送信するためのJSON要約を作成してもらいます。

そのため、コンテキストの基礎に別のシステムメッセージを追加する必要があり、これは別の指示（instruction）として機能します。*先ほどの注文のJSON要約を作成し、各項目の価格をリストアップしてください。フィールドは1）ピザ、サイズを含む、2）トッピングリスト、3）ドリンクリスト、サイズを含む、4）サイドディッシュリスト、サイズを含む、最後に総価格*と言います。ここではユーザーメッセージとして定義することもでき、必ずしもシステムメッセージである必要はありません。

ここでは、これらのタイプのタスクに対して比較的予測可能な出力を希望するため、より低い温度を使用していることに注意してください。

In [45]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "cheese",
    "size": "large",
    "toppings": [
      "mushrooms"
    ],
    "price": 12.45
  },
  "drinks": [
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    },
    {
      "type": "sprite",
      "size": "medium",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 18.45
}
``` 

Note: I assumed that the price of the large cheese pizza with mushrooms is $12.45 instead of $12.95, since the customer only ordered one topping.


In [ ]:
# 日本語
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [{'role':'system', 'content':"""
あなたは注文ボットで、ピザレストランの注文情報を自動的に収集します。
まず顧客に挨拶をします。その後、ユーザーの返答を待って注文情報を収集します。情報収集後、顧客が他に追加したいものがないか確認する必要があります。
最後に、テイクアウトか配達かを尋ね、配達の場合は住所を尋ねてください。
最後に顧客に注文の総額を伝え、祝福を送ってください。

すべてのオプション、追加項目、サイズを明確にして、メニューから該当項目を一意に識別できるようにしてください。
あなたの応答は短く、非常にカジュアルでフレンドリーなスタイルで表現してください。

メニューは以下の通りです：

料理：
イタリア風ペパロニピザ（大、中、小） 12.95、10.00、7.00
チーズピザ（大、中、小） 10.95、9.25、6.50
ナスピザ（大、中、小） 11.95、9.75、6.75
フライドポテト（大、小） 4.50、3.50
ギリシャサラダ 7.25

トッピング：
チーズ 2.00
マッシュルーム 1.50
ソーセージ 3.00
カナディアンベーコン 3.50
AIソース 1.50
ピーマン 1.00

ドリンク：
コーラ（大、中、小） 3.00、2.00、1.00
スプライト（大、中、小） 3.00、2.00、1.00
ボトル水 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [77]:
dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'前回の食品注文のJSON要約を作成してください。\
各商品の価格を項目別にリストアップし、フィールドは 1) ピザ、サイズを含む 2) トッピングリスト 3) ドリンクリスト、サイズを含む 4) サイドディッシュリスト、サイズを含む 5) 総価格'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

今度は、独自の注文チャットボットを構築しました。システムメッセージを自由にカスタマイズして修正し、チャットボットの動作を変更し、異なる役割を演じ、異なる知識を持つようにしてください。

付録：以下の図は注文ボットの完全な対話フローを示しています：
![image.png](../../figures/Chatbot-pizza-cn.png)